# 간단한 전이학습 시켜보기

1. 주제를 정한다.
2. 주제에 맞는 태스크를 판단한다.
3. 해당 테스크를 수행하는 모델을 huggingface에서 찾는다.
4. 적절한 데이터셋을 정제하여 준비한다.
5. 3에서 찾은 모델을 로드라여 '간단하게' 전이학습을 진행한다.
6. 추론 결과를 확인한다.

------

# 단어에 대한 이야기 반환

In [1]:
from transformers import pipeline
import pandas as pd
import random

In [2]:
kogpt = pipeline("text-generation", model="skt/ko-gpt-trinity-1.2B-v0.5")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
prompts_df = pd.read_csv('prompt.txt', encoding='utf-8', sep='\t', header=None, dtype=str)
prompts = prompts_df[0].astype(str).tolist()

In [5]:
prompts

["옛날 옛적에 '{word}'(이)라는 이름을 가진 작은 친구가 살고 있었어요. 그 친구는 매일 특별한 모험을 떠나곤 했지요.",
 '{word}은(는) 단순한 물건이 아니었어요. 그에게는 마음이 있었고, 세상을 여행하고 싶어 했답니다. 그러던 어느 날!',
 "숲 속 깊은 곳, 햇빛이 가득한 언덕에 '{word}'이(가) 자라고 있었어요. 그리고 어느 날, 놀라운 일이 벌어졌습니다. ",
 '옛날에 {word}(이)라는 특별한 존재가 있었어요. 그는 사람들에게 중요한 가르침을 주고 싶어 했지요. 이야기는 이렇게 시작됩니다. ',
 '옛날 옛적에 {word}(이)라는 이름을 가진 작은 마을이 있었어요. ',
 '{word}(이)라는 단어로 시작하는 동화를 들려줄게. ',
 '울창한 숲 속, 산새들이 노래하고, 사슴과 노루가 뛰노는 아름다운 곳에서 {word}이(가) 태어났어요. 그 아이는 커서',
 "어느 화창한 봄 날 {word}은(는) 결심했어요! '나는 바다를 보러 갈거야!'. 사실 {word}은(는) 한 번도 바다를 본 적이 없었죠. 그렇게 {word}의 모험이 시작됐어요!",
 "깊은 바다 속에 살고있는 꽃게는 어느 날 {word}이(가) 궁금해졌어요. '도대체 {word}이(가) 무엇일까? 직접 보고싶어!'. "]

In [6]:
word = "하늘"

sent = ''

def fill_word(sentence: str, word: str) -> str:
    return sentence.replace("{word}", word)

s = random.choice(prompts)
sent = fill_word(s, word)

print(sent)

prompt = f"{sent}\n\n이 문장을 시작으로 동화를 이어 써라."


깊은 바다 속에 살고있는 꽃게는 어느 날 하늘이(가) 궁금해졌어요. '도대체 하늘이(가) 무엇일까? 직접 보고싶어!'. 


In [11]:
import random


# 4) 동화 생성 함수
def make_fairytale(word: str, max_new_tokens: int = 220, seed: int | None = None) -> dict:

    sent = ''

    # 앞 문장 임의 선택
    def fill_word(sentence: str, word: str) -> str:
        return sentence.replace("{word}", word)

    s = random.choice(prompts)
    sent = fill_word(s, word)

    instruction = (
        "위 문장을 첫 문장으로 삼아, 초등학생이 읽기 쉬운 한국어 동화를 "
        "8~12문장으로 이어서 써줘. 따뜻한 결말로 마무리하고, 문장 반복은 피해."
    )
    # prompt = f"{sent}\n\n{instruction}"

    gen = kogpt(sent, max_new_tokens=max_new_tokens, return_full_text=False, no_repeat_ngram_size=3
        # do_sample=True,
        # temperature=0.8,                 # 창의성 (0.7~1.0에서 조절)
        # top_p=0.9,                       # 누적 확률 샘플링
        # repetition_penalty=1.1,          # 반복 억제
        # no_repeat_ngram_size=3,          # 3-그램 반복 방지
    )[0]["generated_text"].strip()

    return {"first_sentence": sent, "story": f"{gen}"}

# 5) 사용 예
result = make_fairytale(word='하늘', max_new_tokens=200, seed=42)
print("첫 문장:", result["first_sentence"])
print("\n=== 생성된 동화 ===\n", result["story"])


첫 문장: 어느 화창한 봄 날 하늘은(는) 결심했어요! '나는 바다를 보러 갈거야!'. 사실 하늘은(는) 한 번도 바다를 본 적이 없었죠. 그렇게 하늘의 모험이 시작됐어요!

=== 생성된 동화 ===
 바다에서 헤엄쳐 나온 하늘이는 친구들에게 바다에서 온 사람이라고 말했어요. 바다에서 모험을 하면서 하늘이는 바다에서 함께 놀 친구가 필요했어요. 하지만 하늘이는 그 친구를 찾을 수 없었어요. 하늘이는 헤엄치는 것도 힘들어하고, 바다에서 혼자 놀았어요. 바다에서는 혼자 놀아야만 했죠. 하늘이는 너무 심심했어요. 친구들은 하늘이가 외로울 거라고 생각했어요. 하늘이는 친구들이 있는 곳으로 갔어요. 하늘이와 친구들이 함께 있었어요. 하늘이가 바다에서 처음 만난 친구들은 모두 바다를 본 적 없는 친구들이었죠. 하늘이의 친구들은 바다를 본적도 없어요. 하늘이는 바닷속에서 나온 지 얼마 되지 않아 친구들과 친해졌어요. 바다 속에서 하늘이는 친구를 알아보았죠. 바다 속에서 헤엄친 지 얼마 되지도 않아 바다를 본 친구들을 만났어요. 하늘이의 친구들이 모두 하늘이를 알아보았어요. 하늘이를 찾았다는 건 하늘이가 친구를 찾았다는 뜻이었죠. 하지만 하늘이가 찾은 친구들은 바다에서 바다로 헤엄치지 않았어요. 바다 속에서는 하늘이가 친구들을 찾을 수 있었거든요. 친구


----------------------------

In [ ]:
# def fill_word(sentence: str, word: str) -> str:
#         return sentence.replace("{word}", word)
# s = random.choice(prompts)
# word = "하늘"
# sent = fill_word(s, word)
# print(sent)

In [ ]:
# instruction = (
#         "위 문장을 첫 문장으로 삼아, 초등학생이 읽기 쉬운 한국어 동화를 "
#         "8~12문장으로 이어서 써줘. 따뜻한 결말로 마무리하고, 문장 반복은 피해."
# )

# prompt = f"{sent}\n\n{instruction}"

# gen = kogpt(sent, max_new_tokens=500, return_full_text=False)[0]["generated_text"].strip()

In [ ]:
# print("첫 문장:", sent)
# print("\n=== 생성된 동화 ===\n", gen)